In [42]:
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
import pandas as pd

In [81]:
df=pd.read_csv('Vic_petition_data.csv',encoding="utf-8")

In [82]:
import numpy as np
from sklearn.model_selection import train_test_split
x, y = (df[['Title', 'Text','Title_str_len' ,
       'start_time', 'Text_len_p1','Text_len_p2', 'Title_len', 'Text_str_len','Image','Tweet']], 
         df.No_Supporters_log)
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.25, random_state=42)

In [48]:
from sklearn.feature_extraction.text import CountVectorizer

text_court_vectorizer = CountVectorizer(analyzer = "word",   
                             tokenizer = None,
                             ngram_range=(1, 2),
                             preprocessor = None, 
                             stop_words = None,   
                             max_features = 2500) 

In [97]:

train_text_features = text_court_vectorizer.fit_transform((x_train.Text).values.astype('U'))
train_text_features = train_text_features.toarray()
test_text_features = text_court_vectorizer.transform((x_test.Text).values.astype('U'))
test_text_features = test_text_features.toarray()


import pickle
pickle.dump(text_court_vectorizer,open('text_court_vectorizer_v2.pickle','wb'))

In [50]:
title_court_vectorizer = CountVectorizer(analyzer = "word",   
                             tokenizer = None,
                             ngram_range=(1, 3),
                             preprocessor = None, 
                             stop_words = None,   
                             max_features = 2500) 

In [98]:
train_title_features = title_court_vectorizer.fit_transform((x_train.Title).values.astype('U'))
train_title_features = train_title_features.toarray()
test_title_features = title_court_vectorizer.transform((x_test.Title).values.astype('U'))
test_title_features = test_title_features.toarray()

#import pickle
pickle.dump(title_court_vectorizer,open('title_court_vectorizer_v2.pickle','wb'))

In [83]:
from sklearn.preprocessing import StandardScaler
#model_tSNE_all = TSNE(n_components=2, random_state=0)
selected_features=['Title_str_len', 'Text_len_p1', 'Text_len_p2',
                                      'Title_len', 'Text_str_len','Image','Tweet']
total_scale_train=StandardScaler().fit(np.concatenate((train_text_features,
                                                     train_title_features,x_train[selected_features]),axis=1))

total_train_scale=total_scale_train.transform(np.concatenate((train_text_features,
                                                     train_title_features,x_train[selected_features]),axis=1))


total_test_scale=total_scale_train.transform(np.concatenate((test_text_features,
                                                             test_title_features, 
                                                x_test[selected_features]),axis=1))

from sklearn.decomposition import PCA
train_pca=PCA(n_components=1500,svd_solver='full')

x_train_pca=train_pca.fit_transform(total_train_scale)

x_test_pca=train_pca.transform(total_test_scale)



In [96]:
import pickle
pickle.dump(total_scale_train,open('total_scale_train_v2.pickle','wb'))
pickle.dump(train_pca,open('train_pca.pickle_v2','wb'))

In [54]:
from sklearn import linear_model, decomposition, datasets
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [84]:
Elastic = linear_model.ElasticNet()
pca = decomposition.PCA()
pipe = Pipeline(steps=[
    ('pca', pca), 
                       ('Elastic', Elastic)])

n_components = [ 60,100,500,750, 1000,]
alphas = np.logspace(-4, 4, 4)
l1_ratio=( 0.2,0.4,0.6,0.7,0.8,1)

estimator = GridSearchCV(pipe,
                         dict(pca__n_components=n_components,
                              Elastic__alpha=alphas,Elastic__l1_ratio=l1_ratio))



In [85]:
estimator.fit(x_train_pca, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(steps=[('pca', PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('Elastic', ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'pca__n_components': [60, 100, 500, 750, 1000], 'Elastic__alpha': array([  1.00000e-04,   4.64159e-02,   2.15443e+01,   1.00000e+04]), 'Elastic__l1_ratio': (0.2, 0.4, 0.6, 0.7, 0.8, 1)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [87]:
n_comp=estimator.best_estimator_.named_steps['pca'].n_components
alpha=estimator.best_estimator_.named_steps['Elastic'].alpha
l1=estimator.best_estimator_.named_steps['Elastic'].l1_ratio


print(n_comp,alpha,l1)


1000 0.0464158883361 0.6


In [88]:
Elastic=linear_model.ElasticNet(alpha=alpha, 
                    l1_ratio=l1, fit_intercept=True, 
            normalize=False, precompute=False, max_iter=1000, 
           copy_X=True, tol=0.0001, warm_start=False, positive=False, 
           random_state=None, selection='cyclic')

In [89]:
train_Elastic=Elastic.fit(x_train_pca[:,0:n_comp],y_train)

In [95]:
import pickle
pickle.dump(train_Elastic,open('train_Elastic_v2.pickle','wb'))

In [90]:
train_predict=train_Elastic.predict(x_train_pca[:,0:n_comp])

In [91]:
from sklearn.metrics import mean_squared_error
MSE_train=mean_squared_error(y_train, train_predict);MSE_train

0.57806138559572984

In [92]:
test_predict=train_Elastic.predict(x_test_pca[:,0:n_comp])


In [93]:
MSE_test=mean_squared_error(y_test, test_predict); MSE_test

0.8859208285978879

In [94]:
from sklearn.metrics import r2_score
r2_score(y_test, test_predict)

0.22412212700140033

In [38]:
print(np.mean(test_predict), np.mean(y_test))

3.86483976102 3.81526681852
